In [420]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from random import sample
# import tensorflow as tf

from torch.utils.data import Dataset
from torch.utils.data.dataset import T_co

from torch import LongTensor


class TargetData(Dataset):
    def __init__(self, num_negatives=4):
        self.num_jobs = 0
        self.df = self.load_ratings()
        self.num_users = self.df.uid.nunique()
        self.num_movies = self.df.mid.nunique()
        self.users = set(self.df.uid.unique())
        self.movies = set(self.df.mid.unique())

        self.train, self.test = self._train_test_split()

        self.training_data = self.add_negatives(self.train, items=self.movies,  n_samples=num_negatives)
        self.testing_data = self.add_negatives(self.test, items=self.movies, n_samples=100)

        self.testing_tensors = self.parse_testing(self.testing_data)

    def __len__(self):
        return self.training_data.shape[0]  # Length of the data to train on

    def __getitem__(self, index) -> T_co:
        user = LongTensor([self.training_data.uid.iloc[index]])  # -1 so that indexing starts from 0
        movie = LongTensor([self.training_data.mid.iloc[index]])
        output = LongTensor([self.training_data.rating.iloc[index]])
        return user, movie, output

    def __call__(self, test_data):
        return self.parse_testing(self.add_negatives(test_data, items=self.movies, n_samples=100))

    @staticmethod
    def load_ratings(min_ratings=5):
        df = pd.read_csv('MovieLens/ratings.dat',
                         sep='::',
                         header=None,
                         names=['uid_old', 'mid_old', 'rating', 'date'],
                         parse_dates=['date'],
                         date_parser=lambda x: pd.to_datetime(x, unit='s', origin='unix'),
                         engine='python')

        # DROP MOVIES WITH LESS THAN 5 RATINGS
        s = df.groupby(['mid_old']).size()
        low_n_ratings = s[s < min_ratings].reset_index().mid_old.tolist()
        df = df[~df.mid_old.isin(low_n_ratings)]
        # RE-INDEX USERS AND MOVIES
        user_id = df[['uid_old']].drop_duplicates().reindex()
        user_id['uid'] = np.arange(len(user_id))
        df = pd.merge(df, user_id, on=['uid_old'], how='left')

        item_id = df[['mid_old']].drop_duplicates()
        item_id['mid'] = np.arange(len(item_id))
        return pd.merge(df, item_id, on=['mid_old'], how='left')

    @staticmethod
    def parse_testing(df):
        test = df.sort_values(by=['uid', 'rating'], ascending=False)
        users, movies, outputs = [], [], []
        for _, u in test.groupby('uid'):
            users.append(LongTensor([u.uid.values]))
            movies.append(LongTensor([u.mid.values]))
            outputs.append(LongTensor([u.rating.values]))
        return users, movies, outputs

    def _train_test_split(self):
        self.df.rating = np.int8(1)
        self.df['latest'] = self.df.groupby(['uid'])['date'].rank(method='first', ascending=False)
        test_bool = self.df.latest == 1
        test = self.df[test_bool]
        train = self.df[~test_bool]
        return (train[['uid', 'mid', 'rating']],
                test[['uid', 'mid', 'rating']]
                )

    def add_negatives(self, df: pd.DataFrame, item: str = 'mid', items=None, n_samples: int = 4):
        if items is None:
            items = set(self.train[item].unique())

        combine = df.groupby('uid')[item].apply(set).reset_index()
        combine['negatives'] = combine[item].apply(lambda x: sample(list(items - x), n_samples))

        s = combine.apply(lambda x: pd.Series(x.negatives, dtype=np.int16), axis=1).stack().reset_index()
        s.rename(columns={'level_0': 'uid', 0: item}, inplace=True)
        s.drop(['level_1'], axis=1, inplace=True)
        s['rating'] = np.int8(0)
        s.uid = s.uid.astype(np.int16)

        complete = pd.concat([df, s]).sort_values(by=['uid', item])
        return complete.reset_index(drop=True)


class AttributeData(Dataset):
    def __init__(self, num_negatives: int = 4, training_ratio: float = .8):
        self.targets = TargetData()
        self.df = pd.merge(
            self._features(),
            self.targets.train,
            # on=['uid', 'uid'],
            how='left'
            )[['uid', 'mid', 'age', 'gender', 'job', 'rating']]

        # self.df.rename(columns={'rating_x': 'rating'}, inplace=True)

        self.num_users = self.df.uid.nunique()
        self.num_jobs = self.df.job.nunique()
        self.jobs = set(self.df.job.unique())
        self.train, self.test = self._train_test_split()

        self.training_data = self.add_negatives(
            self.train,
            item='job',
            items=self.jobs,
            n_samples=num_negatives)
        self.training_data['age'] = self.training_data.groupby('uid')['age'].transform('first')
        self.training_data['gender'] = self.training_data.groupby('uid')['gender'].transform('first')
        self.training_data.dropna(inplace=True)
        # self.training_data['mid'] = self.training_data.apply(lambda x: np.random.choice(1) if x.rating == 0)
        # self.training_data['job'] = self.training_data.groupby('uid')['job'].transform('first')
        # self.jobs_train, self.genders_train, self.ages_train = self.perturb_input(self.training_data)

    def __len__(self):
        return self.train.shape[0]  # Length of the data to train on

    def __getitem__(self, index) -> T_co:
        # features = LongTensor(self.training_data.iloc[index, 1:-1])
        user = LongTensor(self.training_data.uid.iloc[index])
        # job = LongTensor(self.training_data.iloc[index, :])
        # gender = LongTensor(self.training_data.iloc[index, :])
        # age = LongTensor(self.training_data.iloc[index])
        rating = LongTensor(self.training_data.rating.iloc[index])
        return user, rating

    # def __getitem__(self, index) -> T_co:
    #     user = LongTensor(self.training_data.uid.iloc[index])
    #     job = LongTensor(self.training_data.job.iloc[index])
    #     # protected attribute
    #     gender = LongTensor(self.training_data.gender.iloc[index])
    #     age = LongTensor(self.training_data.age.iloc[index])
    #     rating = LongTensor(self.training_data.rating.iloc[index])
    #     return user, job, gender, age, rating

    # def __call__(self, test_data):
    #     return self.parse_testing(self.add_negatives(test_data, items=self.movies, n_samples=100))

    def perturb_input(self, df):
        # HOT ENCODING (CATEGORICAL)
        func1, func2 = self.obfuscation_functions()

        jobs_train = pd.get_dummies(df.job, drop_first=True)

        genders_train = pd.get_dummies(df.gender, drop_first=True)

        # (CONTINUOUS)
        ages_train = 2 * ((df.age - df.age.min()) /
                          (df.age.max() - df.age.min())) - 1
        return jobs_train.apply(func2), genders_train.apply(func2), ages_train.apply(func1)

    @staticmethod
    def obfuscation_functions(eps_hat: int = 4):
        n_features = 3  # d

        delta = n_features + n_features ** 2 / 4  # global sensitivity
        # slack = np.random.uniform(0, 1, 1)
        # slack = np.max(1, n_features, np.int8(local_epsilon / 2.5))

        C = (np.exp(eps_hat / 2) + 1) / (np.exp(eps_hat / 2) - 1)

        l = lambda x: ((C + 1) / 2) * x - ((C - 1) / 2)
        pi = lambda x: l(x) + C - 1

        dfrac = np.exp(eps_hat / 2) / (np.exp(eps_hat / 2) + 1)
        const = lambda x: 1 / (np.exp(eps_hat / x) + 1)

        def func1(x):
            if np.random.uniform(0, 1, 1) < dfrac:
                return np.random.uniform(l(x), pi(x), 1)
            else:
                return np.random.choice(
                    [np.random.uniform(-C, l(x), 1),
                     np.random.uniform(pi(x), C, 1).squeeze(0)], 1
                )
        def func2(x):
            array = []
            for i in x:
                if i == 1:
                    array.append(np.float32(.5))
                else:
                    array.append(const(np.random.uniform(0, 1, 1)))
            return array
        return func1, func2

    def add_negatives(self, df: pd.DataFrame, item: str = 'mid', items=None, n_samples: int = 4):
        if items is None:
            items = set(self.df[item].unique())

        movies = set(self.df.mid.unique())
        scombine = df.groupby('uid')[item].apply(set).reset_index()
        mcombine = df.groupby('uid')['mid'].apply(set).reset_index()
        scombine['jnegatives'] = scombine[item].apply(lambda x: sample(list(items - x), n_samples))
        mcombine['mnegatives'] = mcombine['mid'].apply(lambda x: sample(list(movies - x), n_samples))

        s = scombine.apply(lambda x: pd.Series(x.jnegatives, dtype=np.int16), axis=1).stack().reset_index()
        m = mcombine.apply(lambda x: pd.Series(x.mnegatives, dtype=np.int16), axis=1).stack().reset_index()
        s.rename(columns={'level_0': 'uid', 0: item}, inplace=True)
        m.rename(columns={'level_0': 'uid', 0: 'mid'}, inplace=True)
        s.drop(['level_1'], axis=1, inplace=True)
        m.drop(['level_1'], axis=1, inplace=True)
        s['rating'] = np.int8(0)
        s.uid = s.uid.astype(np.int16)
        s = pd.merge(s, m, on=['uid', 'uid'], how='inner')
        complete = pd.concat([df, s]).sort_values(by=['uid', item])
        # complete = pd.concat([df, s]).sort_values(by=['uid', item])
        return complete.reset_index(drop=True)

    #
    # def _train_test_split(self):
    #     # self.df['latest'] = self.df.groupby(['uid'])['date'].rank(method='first', ascending=False)
    #     test_bool = self.df.latest <= 1
    #     test = self.df[test_bool]
    #     train = self.df[~test_bool]
    #     return (train[['uid', 'mid', 'age', 'gender', 'job', 'rating']],
    #             test[['uid', 'mid', 'age', 'gender', 'job', 'rating']]
    #             )

    def _train_test_split(self, train_ratio: float = .8):
        msk = np.random.rand(len(self.df)) < train_ratio

        train = self.df[msk]
        test = self.df[~msk]
        return train, test

    def _features(self):
        df = pd.read_csv('MovieLens/users.dat',
                         sep='::',
                         header=None,
                         names=['uid', 'gender', 'age', 'job', 'zip'],
                         engine='python')
        df.drop(columns=['uid'], inplace=True)
        df.index.rename('uid', inplace=True)
        df.gender = pd.get_dummies(df.gender, drop_first=True)  # 0:F, 1:M
        df.reset_index(inplace=True)
        drop = [0, 10, 13, 19]

        clean = df[~df['job'].isin(drop)]

        clean['rating'] = 1
        clean['uid'] = clean.uid - 1

        self.num_jobs = clean.job.nunique()

        item_id = clean[['job']].drop_duplicates()
        item_id['njob'] = np.arange(self.num_jobs)
        clean = pd.merge(clean, item_id, on=['job'], how='left')
        clean.job = clean.njob
        return clean


class DataGenerator(AttributeData):
    def __init__(self):
        super().__init__()

        # self.targets = TargetData()
        # self.full_train = pd.merge(self.targets.training_data, self.df, on=['uid', 'uid'], how='left')
        # self.jobs, self.genders, self.ages = self.perturb_input(self.full_train)

    def __len__(self):
        return self.jobs.shape[0]  # Length of the data to train on


In [421]:
data = AttributeData()

In [422]:
# data.df
data.train[data.train.uid==300]

,uid,mid,age,gender,job,rating
37357,300,218,18,1,10,1
37358,300,1321,18,1,10,1
37359,300,58,18,1,10,1
37360,300,556,18,1,10,1
37362,300,73,18,1,10,1
...,...,...,...,...,...,...
37708,300,217,18,1,10,1
37709,300,1017,18,1,10,1
37710,300,837,18,1,10,1
37711,300,783,18,1,10,1


In [423]:
data.training_data[data.training_data.uid==300]

,uid,mid,age,gender,job,rating
34723,300,1160,18.0,1.0,4,0
34724,300,828,18.0,1.0,4,0
34725,300,1962,18.0,1.0,4,0
34726,300,2252,18.0,1.0,4,0
34727,300,1160,18.0,1.0,9,0
...,...,...,...,...,...,...
35024,300,2252,18.0,1.0,11,0
35025,300,1160,18.0,1.0,15,0
35026,300,828,18.0,1.0,15,0
35027,300,1962,18.0,1.0,15,0


In [424]:
from time import time
import torch
from torch.utils.data import DataLoader
num_epochs = 25
batch_size = 2048
learning_rate = .001
top_k = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [425]:
def train_ncf(model):
    # data.get_train_instances(seed=e)
    optimizer = torch.optim.AdamW(model.parameters(),  lr=learning_rate, weight_decay=1e-6)

    dataloader = DataLoader(data, batch_size=batch_size,
                            shuffle=True, num_workers=0)
    t1 = time()
    it_per_epoch = len(data) / batch_size
    for i in range(num_epochs):
        model.train()
        print("Starting epoch ", i + 1)
        j = 0
        for batch in dataloader:
            # u, j, g, a, r = batch
            u, r = batch
            print(u)
            break
            # move tensors to cuda
            f = f.to(device)
            # m = m.to(device)
            r = r.to(device)
            y_hat = model(f.squeeze(1), r.squeeze(1))
            loss = torch.nn.BCELoss()  # (weight=w, reduction="mean")
            loss = loss(y_hat, r.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if j % int(1 + it_per_epoch / 10) == 0:
                print("Progress: ", round(100 * j / it_per_epoch), "%")
            j+=1

        # Epoch metrics
        t2 = time()
        print("Epoch time:", round(t2 - t1), "seconds")
        print("Loss:", loss / i)
        ncf.eval()


        # print('baseline ')
        # t1 = time()
        # hr, ndcg = evaluate_model(ncf,data.test[['uid', 'mid']].values,10,100, 3416)
        # t2 = time()
        # print("Evaluation time:", round(t2 - t1), "seconds")
        # print(f"HR@{top_k}:{hr[-1]}")
        #
        # print("(evaluator)...")
        # t1 = time()
        # hr, ndcg = evaluator()
        # # hr, ndcg = evaluate(model, data.test, top_K)
        # t2 = time()
        # print("Evaluation time:", round(t2 - t1), "seconds")
        # print(f"HR@{top_k}:{hr}")

        print("Evaluating (eval_model)...")
        t1 = time()
        hr, ndcg = eval_model(model, data)
        t2 = time()
        print("Evaluation time:", round(t2 - t1), "seconds")
        print(f"HR@{top_k}:{hr}")
        #
        #
        # print("Evaluating (tf)...")
        # t1 = time()
        # hr, ndcg = tf_eval(1)
        # t2 = time()
        #
        # print("Evaluation time:", round(t2 - t1), "seconds")
        # print(f"HR@{top_k}:{np.array(hr).mean()}")
        # new
        # HR, NDCG = evaluate_model(model, data, validation=False)
        # updated
        # hr, ndcg = evaluate_model(model, data.test, top_K, random_samples)
        # original
        loss = 0
        print()

    print("Done")

In [426]:
from models import NCF

ncf = NCF(6040, 3952, 128, [128, 64, 32, 16], 1)

ncf.load_state_dict(torch.load('models/preTrained_NCF'))

<All keys matched successfully>

In [427]:
train_ncf(ncf)


Starting epoch  1


RuntimeError: stack expects each tensor to be equal size, but got [526] at entry 0 and [2367] at entry 1

In [ ]:
data.targets.training_data

In [348]:
data.training_data.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 722986 entries, 0 to 737705
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   uid     722986 non-null  int64  
 1   mid     722986 non-null  int64  
 2   age     722986 non-null  float64
 3   gender  722986 non-null  float64
 4   job     722986 non-null  int64  
 5   rating  722986 non-null  int64  
dtypes: float64(2), int64(4)
memory usage: 38.6 MB


In [380]:
x = data.add_negatives(data.test, item='job', items = set(data.df.job.unique()))
x[x.uid==0]

,uid,mid,age,gender,job,rating
0,0,0,56.0,1.0,0,1
1,0,7,56.0,1.0,0,1
2,0,9,56.0,1.0,0,1
3,0,17,56.0,1.0,0,1
4,0,19,56.0,1.0,0,1
5,0,20,56.0,1.0,0,1
6,0,23,56.0,1.0,0,1
7,0,26,56.0,1.0,0,1
8,0,33,56.0,1.0,0,1
9,0,43,56.0,1.0,0,1


In [393]:
def add_negatives(df: pd.DataFrame, item: str = 'mid', n_samples: int = 4):
    movies = set(data.df.mid.unique())
    items = set(data.df.mid.unique())
    scombine = df.groupby('uid')[item].apply(set).reset_index()
    mcombine = df.groupby('uid')['mid'].apply(set).reset_index()
    scombine['jnegatives'] = scombine[item].apply(lambda x: sample(list(items - x), n_samples))
    mcombine['mnegatives'] = mcombine['mid'].apply(lambda x: sample(list(movies - x), n_samples))

    s = scombine.apply(lambda x: pd.Series(x.jnegatives, dtype=np.int16), axis=1).stack().reset_index()
    m = mcombine.apply(lambda x: pd.Series(x.mnegatives, dtype=np.int16), axis=1).stack().reset_index()
    s.rename(columns={'level_0': 'uid', 0: item}, inplace=True)
    m.rename(columns={'level_0': 'uid', 0: 'mid'}, inplace=True)
    s.drop(['level_1'], axis=1, inplace=True)
    m.drop(['level_1'], axis=1, inplace=True)
    s['rating'] = np.int8(0)
    s.uid = s.uid.astype(np.int16)
    s = pd.merge(s, m, on=['uid', 'uid'], how='inner')
    complete = pd.concat([df, s]).sort_values(by=['uid', item])
    # complete = pd.concat([df, s]).sort_values(by=['uid', item])
    return complete.reset_index(drop=True)

In [387]:
data.test

,uid,mid,age,gender,job,rating
4,0,4,56,1,0,1
5,0,5,56,1,0,1
8,0,8,56,1,0,1
14,0,14,56,1,0,1
25,0,26,56,1,0,1
...,...,...,...,...,...,...
823181,6038,421,25,1,16,1
823184,6038,45,25,1,16,1
823186,6038,2046,25,1,16,1
823187,6038,1092,25,1,16,1


In [388]:
data.train

,uid,mid,age,gender,job,rating
0,0,0,56,1,0,1
1,0,1,56,1,0,1
2,0,2,56,1,0,1
3,0,3,56,1,0,1
6,0,6,56,1,0,1
...,...,...,...,...,...,...
823189,6038,1644,25,1,16,1
823190,6038,1795,25,1,16,1
823191,6038,1582,25,1,16,1
823192,6038,419,25,1,16,1


In [391]:
test = pd.merge(
    data._features(),
    data.targets.test,
    # on=['uid', 'uid'],
    how='left'
)[['uid', 'mid', 'age', 'gender', 'job', 'rating']]

In [392]:
test

,uid,mid,age,gender,job,rating
0,0,25,56,1,0,1
1,1,66,25,1,1,1
2,2,207,45,1,2,1
3,3,208,25,1,3,1
4,4,222,50,0,4,1
...,...,...,...,...,...,...
4915,6033,985,25,0,5,1
4916,6034,812,25,0,1,1
4917,6035,1048,45,0,5,1
4918,6036,294,56,0,5,1


In [396]:
full_test = add_negatives(test, item='job', n_samples=100)

In [397]:
full_test['age'] = full_test.groupby('uid')['age'].transform('first')
full_test['gender'] = full_test.groupby('uid')['gender'].transform('first')
full_test.dropna(inplace=True)
full_test

,uid,mid,age,gender,job,rating
0,0,25,56.0,1.0,0,1
1,0,3275,56.0,1.0,56,0
2,0,2790,56.0,1.0,56,0
3,0,691,56.0,1.0,56,0
4,0,1514,56.0,1.0,56,0
...,...,...,...,...,...,...
49204915,6033,985,25.0,0.0,5,1
49204916,6034,812,25.0,0.0,1,1
49204917,6035,1048,45.0,0.0,5,1
49204918,6036,294,56.0,0.0,5,1


In [403]:
def parse_testing(df):
    test = df.sort_values(by=['uid', 'rating'], ascending=False)
    users, features, outputs = [], [], []
    for _, u in test.groupby('uid'):
        users.append(LongTensor([u.uid.values]))
        features.append(LongTensor([u[['mid', 'age', 'gender', 'job']].values]))
        outputs.append(LongTensor([u.rating.values]))
    return users, features, outputs


In [404]:
tensors = parse_testing(full_test)

/var/folders/kf/j1jq_1qs78n4gd1vczc8vfw80000gn/T/ipykernel_97404/423272089.py:6: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  features.append(LongTensor([u[['mid', 'age', 'gender', 'job']].values]))


In [405]:
def rank(l, item):
    # rank of the test item in the list of negative instances
    # returns the number of elements that the test item is bigger than

    index = 0
    for element in l:
        if element > item:
            index += 1
            return index
        index += 1
    return index
def eval_model(model, data, num_users=6040):
    # Evaluates the model and returns HR@10 and NDCG@10
    hits = 0
    ndcg = 0
    for u in range(num_users):
        user = data.testing_tensors[0][u].squeeze().to(device)
        item = data.testing_tensors[1][u].squeeze().to(device)
        y = model(user, item)

        y = y.tolist()
        y = sum(y, [])
        first = y.pop(0)
        y.sort()
        ranking = rank(y, first)
        if ranking > 90:
            hits += 1
            ndcg += np.log(2) / np.log(len(user) - ranking + 1)

    hr = hits / num_users
    ndcg = ndcg / num_users
    return hr, ndcg

In [406]:
tensors

([tensor([[0, 0, 0,  ..., 0, 0, 0]]),
  tensor([[1, 1, 1,  ..., 1, 1, 1]]),
  tensor([[2, 2, 2,  ..., 2, 2, 2]]),
  tensor([[3, 3, 3,  ..., 3, 3, 3]]),
  tensor([[4, 4, 4,  ..., 4, 4, 4]]),
  tensor([[5, 5, 5,  ..., 5, 5, 5]]),
  tensor([[6, 6, 6,  ..., 6, 6, 6]]),
  tensor([[7, 7, 7,  ..., 7, 7, 7]]),
  tensor([[8, 8, 8,  ..., 8, 8, 8]]),
  tensor([[9, 9, 9,  ..., 9, 9, 9]]),
  tensor([[10, 10, 10,  ..., 10, 10, 10]]),
  tensor([[11, 11, 11,  ..., 11, 11, 11]]),
  tensor([[13, 13, 13,  ..., 13, 13, 13]]),
  tensor([[15, 15, 15,  ..., 15, 15, 15]]),
  tensor([[16, 16, 16,  ..., 16, 16, 16]]),
  tensor([[18, 18, 18,  ..., 18, 18, 18]]),
  tensor([[19, 19, 19,  ..., 19, 19, 19]]),
  tensor([[20, 20, 20,  ..., 20, 20, 20]]),
  tensor([[22, 22, 22,  ..., 22, 22, 22]]),
  tensor([[23, 23, 23,  ..., 23, 23, 23]]),
  tensor([[24, 24, 24,  ..., 24, 24, 24]]),
  tensor([[25, 25, 25,  ..., 25, 25, 25]]),
  tensor([[26, 26, 26,  ..., 26, 26, 26]]),
  tensor([[27, 27, 27,  ..., 27, 27, 27]]),
  te